<a href="https://colab.research.google.com/github/kowsiknd/MNIST-PyTorch/blob/main/MNIST_Convolution_Neural_Network_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [2]:
class CNN(nn.Module):
  def __init__(self, in_channels=1, num_class=10):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) # Same
    self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
    
    self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) # Same
    self.fc1 = nn.Linear(16*7*7, num_class)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    x = x.reshape(x.shape[0], -1)
    x = self.fc1(x)
    return x

In [3]:
model = CNN()

In [4]:
x = torch.randn(64, 1, 28, 28)
print(model(x).shape)

torch.Size([64, 10])


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [17]:
in_channels = 1
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epoch = 10
load_model = True

In [18]:
train_dataset = datasets.MNIST(root='/dataset', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='/dataset', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [19]:
model = CNN(
    in_channels=in_channels,
    num_class=num_classes
).to(device)

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=learning_rate
)

In [21]:
def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
  print("=>Saving Checkpoint")
  torch.save(state, filename)

In [22]:
def load_checkpoint(checkpoint):
  print("=> Load checkpoint")
  # print(checkpoint)
  model.load_state_dict(checkpoint['state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer'])

In [23]:
if load_model:
  load_checkpoint(torch.load("my_checkpoint.pth.tar"))

=> Load checkpoint


In [24]:
for epoch in range(num_epoch):
  losses = []

  if epoch%2 == 0:
    check_point = {'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict()}
    save_checkpoint(check_point)

  for batch_idx, (data, targets) in enumerate(train_loader):
    data = data.to(device=device)
    targets = targets.to(device=device)

    scores = model(data)
    loss = criterion(scores, targets)
    losses.append(loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

=>Saving Checkpoint
=>Saving Checkpoint
=>Saving Checkpoint
=>Saving Checkpoint
=>Saving Checkpoint


In [25]:
def check_accuracy(loader, model):
  if loader.dataset.train:
    print("Checking accuracy on training data")
  else:
    print("Checking accuracy on test data")

  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device=device)
      y = y.to(device=device)

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

    print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
  
  model.train()

In [26]:
check_accuracy(train_loader, model)

Checking accuracy on training data
Got 59788 / 60000 with accuracy 99.65


In [27]:
check_accuracy(test_loader, model)


Checking accuracy on test data
Got 9869 / 10000 with accuracy 98.69
